# 6. Understanding ROS Topics
http://wiki.ros.org/ROS/Tutorials/UnderstandingTopics

In [1]:
import os
import sys

home_dir = os.getcwd()
home_dir

'/home/pi/notebooks/Tutorials/1_Core_ROS_Tutorials'

In [2]:
# get ROS related environment variables

envs = !printenv | grep ROS 
ROS_ENVs = {}

for env in envs:
    env = env.split('=')
    [k, v] = env
    ROS_ENVs[k] = v

ROS_ENVs['LD_LIBRARY_PATH'] = '/opt/ros/kinetic/lib'
ROS_ENVs['PKG_CONFIG_PATH'] = '/opt/ros/kinetic/lib/pkgconfig'
ROS_ENVs['CMAKE_PREFIX_PATH'] = '/opt/ros/kinetic'  

## Setup
### roscore
Let's start by making sure that we have roscore running:  

open a new terminal window, run a ROS Docker container with:
```
docker run -it --rm \
	--net=host \
	-v /tmp/.X11-unix:/tmp/.X11-unix \
	-e DISPLAY=unix$DISPLAY \
	--cap-add=SYS_ADMIN \
	--device /dev/snd \
	--device /dev/dri \
	-v /dev/shm:/dev/shm \
	-p 11311:11311 \
	wei1234c/ros_ubuntu_amd64
```

in the container, run roscore:
```
$ roscore
```

### turtlesim
For this tutorial we will also use turtlesim.  
open a new terminal window, run a ROS Docker container with:
```
docker run -it --rm \
	--net=host \
	-v /tmp/.X11-unix:/tmp/.X11-unix \
	-e DISPLAY=unix$DISPLAY \
	--cap-add=SYS_ADMIN \
	--device /dev/snd \
	--device /dev/dri \
	-v /dev/shm:/dev/shm \
	-p 11311:11311 \
	wei1234c/ros_ubuntu_amd64
```

in the container, run:
```
$ rosrun turtlesim turtlesim_node
```

### turtle keyboard teleoperation
We'll also need something to drive the turtle around with.  

open a new terminal window, run a ROS Docker container with:
```
docker run -it --rm \
	--net=host \
	-v /tmp/.X11-unix:/tmp/.X11-unix \
	-e DISPLAY=unix$DISPLAY \
	--cap-add=SYS_ADMIN \
	--device /dev/snd \
	--device /dev/dri \
	-v /dev/shm:/dev/shm \
	-p 11311:11311 \
	wei1234c/ros_ubuntu_amd64
```

in the container, run:
```
$ rosrun turtlesim turtle_teleop_key
```

Now you can use the arrow keys of the keyboard to drive the turtle around. If you can not drive the turtle, **select the terminal window of the turtle_teleop_key** to make sure that the keys that you type are recorded.

## ROS Topics
The turtlesim_node and the turtle_teleop_key node are communicating with each other over a ROS Topic. turtle_teleop_key is publishing the key strokes on a topic, while turtlesim subscribes to the same topic to receive the key strokes. Let's use rqt_graph which shows the nodes and topics currently running.

Note: If you're using electric or earlier, rqt is not available. Use rxgraph instead.

## Using rqt_graph
rqt_graph creates a dynamic graph of what's going on in the system. rqt_graph is part of the rqt package. Unless you already have it installed,   

open a new terminal window, run a ROS Docker container with:
```
docker run -it --rm \
	--net=host \
	-v /tmp/.X11-unix:/tmp/.X11-unix \
	-e DISPLAY=unix$DISPLAY \
	--cap-add=SYS_ADMIN \
	--device /dev/snd \
	--device /dev/dri \
	-v /dev/shm:/dev/shm \
	-p 11311:11311 \
	wei1234c/ros_ubuntu_amd64
```

in the container, run: 
```
$ sudo apt-get install -y ros-kinetic-rqt ros-kinetic-rqt-common-plugins
```

then, run: 
```
$ rosrun rqt_graph rqt_graph  
```

You will see something similar to:

<img src='http://wiki.ros.org/ROS/Tutorials/UnderstandingTopics?action=AttachFile&do=get&target=rqt_graph_turtle_key.png'>

If you place your mouse over /turtle1/command_velocity it will highlight the ROS nodes (here blue and green) and topics (here red). As you can see, the turtlesim_node and the turtle_teleop_key nodes are communicating on the topic named /turtle1/command_velocity.

<img src='http://wiki.ros.org/ROS/Tutorials/UnderstandingTopics?action=AttachFile&do=get&target=rqt_graph_turtle_key2.png'>

## Introducing rostopic
The **rostopic** tool allows you to get information about ROS topics.

You can use the help option to get the available sub-commands for rostopic 

In [3]:
!rostopic -h

rostopic is a command-line tool for printing information about ROS Topics.

Commands:
	rostopic bw	display bandwidth used by topic
	rostopic delay	display delay of topic from timestamp in header
	rostopic echo	print messages to screen
	rostopic find	find topics by type
	rostopic hz	display publishing rate of topic    
	rostopic info	print information about active topic
	rostopic list	list active topics
	rostopic pub	publish data to topic
	rostopic type	print topic or field type

Type rostopic <command> -h for more detailed usage, e.g. 'rostopic echo -h'



## Using rostopic echo
**rostopic echo** shows the data published on a topic.

Usage:
```
rostopic echo [topic]
```

In [4]:
!rostopic echo /turtle1/cmd_vel

linear: 
  x: 2.0
  y: 0.0
  z: 0.0
angular: 
  x: 0.0
  y: 0.0
  z: 0.0
---
linear: 
  x: 0.0
  y: 0.0
  z: 0.0
angular: 
  x: 0.0
  y: 0.0
  z: -2.0
---
linear: 
  x: -2.0
  y: 0.0
  z: 0.0
angular: 
  x: 0.0
  y: 0.0
  z: 0.0
---
^C


Now let's look at rqt_graph again. Press the refresh button in the upper-left to show the new node. As you can see rostopic echo, shown here in red, is now also subscribed to the turtle1/command_velocity topic.  
<img src='http://wiki.ros.org/ROS/Tutorials/UnderstandingTopics?action=AttachFile&do=get&target=rqt_graph_echo.png'>

## Using rostopic list
rostopic list returns a list of all topics currently subscribed to and published.

Let's figure out what argument the list sub-command needs. In a new terminal run:

In [5]:
!rostopic list -h

Usage: rostopic list [/namespace]

Options:
  -h, --help            show this help message and exit
  -b BAGFILE, --bag=BAGFILE
                        list topics in .bag file
  -v, --verbose         list full details about each topic
  -p                    list only publishers
  -s                    list only subscribers
  --host                group by host name


In [6]:
!rostopic list -v


Published topics:
 * /turtle1/color_sensor [turtlesim/Color] 1 publisher
 * /turtle1/cmd_vel [geometry_msgs/Twist] 1 publisher
 * /rosout [rosgraph_msgs/Log] 3 publishers
 * /rosout_agg [rosgraph_msgs/Log] 1 publisher
 * /turtle1/pose [turtlesim/Pose] 1 publisher

Subscribed topics:
 * /turtle1/cmd_vel [geometry_msgs/Twist] 1 subscriber
 * /rosout [rosgraph_msgs/Log] 1 subscriber
 * /statistics [rosgraph_msgs/TopicStatistics] 1 subscriber



## ROS Messages
Communication on topics happens by sending ROS messages between nodes. For the publisher (turtle_teleop_key) and subscriber (turtlesim_node) to communicate, the publisher and subscriber must send and receive the same **type** of message. This means that a topic **type** is defined by the message **type** published on it. The **type** of the message sent on a topic can be determined using rostopic **type**.

### Using rostopic **type**
rostopic **type** returns the message **type** of any topic being published.

Usage:
```
rostopic type [topic]
```

In [7]:
!rostopic type /turtle1/cmd_vel

geometry_msgs/Twist


We can look at the details of the message using rosmsg:

In [8]:
!rosmsg show geometry_msgs/Twist

geometry_msgs/Vector3 linear
  float64 x
  float64 y
  float64 z
geometry_msgs/Vector3 angular
  float64 x
  float64 y
  float64 z



## rostopic continued
Now that we have learned about ROS messages, let's use rostopic with messages.

### Using rostopic pub
rostopic pub publishes data on to a topic currently advertised.

Usage:
```
rostopic pub [topic] [msg_type] [args]
```

In [9]:
!rostopic pub -1 /turtle1/cmd_vel geometry_msgs/Twist -- '[2.0, 0.0, 0.0]' '[0.0, 0.0, 1.8]'

publishing and latching message for 3.0 seconds


In [17]:
!rostopic pub /turtle1/cmd_vel geometry_msgs/Twist -r 1 -- '[2.0, 0.0, 0.0]' '[0.0, 0.0, -1.8]'

^C


We can also look at what is happening in rqt_graph. Press the refresh button in the upper-left. The rostopic pub node (here in red) is communicating with the rostopic echo node (here in green):

<img src='http://wiki.ros.org/ROS/Tutorials/UnderstandingTopics?action=AttachFile&do=get&target=rqt_graph_pub.png'>

### Using rostopic hz
rostopic hz reports the rate at which data is published.

Usage:
```
rostopic hz [topic]
```

Let's see how fast the turtlesim_node is publishing /turtle1/pose:



In [14]:
!rostopic hz /turtle1/pose

subscribed to [/turtle1/pose]
average rate: 62.486
	min: 0.015s max: 0.017s std dev: 0.00040s window: 60
average rate: 62.503
	min: 0.015s max: 0.017s std dev: 0.00045s window: 123
average rate: 62.509
	min: 0.015s max: 0.017s std dev: 0.00047s window: 186
average rate: 62.501
	min: 0.015s max: 0.017s std dev: 0.00048s window: 248
average rate: 62.494
	min: 0.015s max: 0.018s std dev: 0.00048s window: 311
average rate: 62.503
	min: 0.014s max: 0.018s std dev: 0.00050s window: 373
average rate: 62.499
	min: 0.014s max: 0.018s std dev: 0.00049s window: 436
average rate: 62.496
	min: 0.014s max: 0.018s std dev: 0.00048s window: 499
average rate: 62.502
	min: 0.014s max: 0.018s std dev: 0.00048s window: 561
average rate: 62.499
	min: 0.014s max: 0.018s std dev: 0.00048s window: 624
average rate: 62.498
	min: 0.014s max: 0.018s std dev: 0.00047s window: 686
average rate: 62.502
	min: 0.014s max: 0.018s std dev: 0.00047s window: 749
average rate: 62.501
	min: 0.014s max: 0.018s std dev: 0.00

 use rostopic type in conjunction with rosmsg 

In [15]:
!rostopic type /turtle1/cmd_vel | rosmsg show

geometry_msgs/Vector3 linear
  float64 x
  float64 y
  float64 z
geometry_msgs/Vector3 angular
  float64 x
  float64 y
  float64 z



## Using rqt_plot
Note: If you're using electric or earlier, rqt is not available. Use rxplot instead.

rqt_plot displays a scrolling time plot of the data published on topics. Here we'll use rqt_plot to plot the data being published on the /turtle1/pose topic. First:  

open a new terminal window, run a ROS Docker container with:
```
docker run -it --rm \
	--net=host \
	-v /tmp/.X11-unix:/tmp/.X11-unix \
	-e DISPLAY=unix$DISPLAY \
	--cap-add=SYS_ADMIN \
	--device /dev/snd \
	--device /dev/dri \
	-v /dev/shm:/dev/shm \
	-p 11311:11311 \
	wei1234c/ros_ubuntu_amd64
```

start rqt_plot by typing:
```
$ rosrun rqt_plot rqt_plot
```

in a new terminal. In the new window that should pop up, a text box in the upper left corner gives you the ability to add any topic to the plot.   

Typing **/turtle1/pose/x** will highlight the plus button, previously disabled.  

Press it and repeat the same procedure with the topic **/turtle1/pose/y**. You will now see the turtle's x-y location plotted in the graph.

<img src='http://wiki.ros.org/ROS/Tutorials/UnderstandingTopics?action=AttachFile&do=get&target=rqt_plot.png'>

Pressing the minus button shows a menu that allows you to hide the specified topic from the plot. Hiding both the topics you just added and adding **/turtle1/pose/theta** will result in the plot shown in the next figure.

<img src='http://wiki.ros.org/ROS/Tutorials/UnderstandingTopics?action=AttachFile&do=get&target=rqt_plot2.png'>

## Video Tutorial
https://www.youtube.com/watch?v=Yx_vGAt74sk